# Deep Learning - Perceptron with Regularization
Aluno: Lucas Mendes Massa

Implement a perceptron with and without regularization. Test the accuracy of both variations of the perceptron on both the training data and the out-of-sample data on the Ionosphere data set of the UCI Machine Learning Repository. What do you observe about the effect of regularization in the two cases? Repeat the experiment with smaller samples of the Ionosphere training data, and report your  observations.

## Lendo a base e separando em datasets de treino e teste

In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [7]:
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv'
df = pd.read_csv(url, header=None)

In [8]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,g
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,b
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,g
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,b
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,g


In [104]:
X, y = df.values[:, :-1], df.values[:, -1]
X = X.astype('float32')
y = LabelEncoder().fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Perceptron sem regularizacao

In [21]:
class Perceptron:
    def __init__(self, number_of_features: int, learning_rate: float, epochs: int):
        self.weights = None
        self.bias = None
        self.number_of_features = number_of_features
        self.learning_rate = learning_rate
        self.epochs = epochs
        
    def reset_before_fit(self):
        self.weights = np.zeros((self.number_of_features))
        self.bias = 0
        
    def activation(self, z: float) -> float:
        return np.heaviside(z, 0)
    
    def fit(self, X: np.ndarray, y: np.ndarray) -> None:        
        self.reset_before_fit()        
        for epoch in tqdm(range(self.epochs)):            
            for i in range(len(X)):
                z = np.dot(X, self.weights) + self.bias
                y_pred = self.activation(z)
                self.weights = self.weights + self.learning_rate * (y[i] - y_pred[i]) * X[i] # update weights
                self.bias = self.bias + self.learning_rate * (y[i] - y_pred[i]) # update bias                
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        z = np.dot(X, self.weights) + self.bias
        return self.activation(z)

In [121]:
model1 = Perceptron(X.shape[1], 0.01, 2000)
model1.fit(X_train, y_train)

100%|██████████| 2000/2000 [00:08<00:00, 236.86it/s]


In [122]:
y_train_pred = model1.predict(X_train)
y_test_pred = model1.predict(X_test)

In [123]:
accuracy_score(y_train, y_train_pred)

0.9591836734693877

In [124]:
accuracy_score(y_test, y_test_pred)

0.839622641509434

## Perceptron com regularizacao

In [99]:
class RegularizedPerceptron:
    def __init__(self, number_of_features: int, learning_rate: float, epochs: int, reg_weight: int):
        self.weights = None
        self.bias = None
        self.number_of_features = number_of_features
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.reg_weight = reg_weight
        
    def reset_before_fit(self):
        self.weights = np.zeros((self.number_of_features))
        self.bias = 0
        
    def activation(self, z: float) -> float:
        return np.heaviside(z, 0)
    
    def fit(self, X: np.ndarray, y: np.ndarray) -> None:        
        self.reset_before_fit()        
        for epoch in tqdm(range(self.epochs)):            
            for i in range(len(X)):
                z = np.dot(X, self.weights) + self.bias
                y_pred = self.activation(z)
                self.weights = self.weights*(1-(2*self.learning_rate*self.reg_weight)/self.number_of_features) \
                                + self.learning_rate * (y[i] - y_pred[i]) * X[i] # update weights
                self.bias = self.bias + self.learning_rate * (y[i] - y_pred[i]) # update bias        
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        z = np.dot(X, self.weights) + self.bias
        return self.activation(z)

In [125]:
model2 = RegularizedPerceptron(X.shape[1], 0.01, 2000, 0.1)
model2.fit(X_train, y_train)

100%|██████████| 2000/2000 [00:09<00:00, 216.03it/s]


In [126]:
y_train_pred = model2.predict(X_train)
y_test_pred = model2.predict(X_test)

In [127]:
accuracy_score(y_train, y_train_pred)

0.9428571428571428

In [128]:
accuracy_score(y_test, y_test_pred)

0.8113207547169812